In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install implicit==0.4.4

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


###Практика:

4) Поэкспериментировать с ALS (grid-search) ****

Подобрать лучшие параметры
Написать отчет об увиденном

`MatrixFactorization` (Матричная факторизация) - это метод, используемый в области рекомендательных систем для поиска скрытых шаблонов или факторов в матрице взаимодействий между пользователями и предметами. Он базируется на представлении этой матрицы в виде произведения двух матриц более низкого ранга.

Этот метод заключается в разложении матрицы взаимодействий (например, матрицы оценок пользователей по предметам) на две более низкоранговые матрицы - матрицу пользователей и матрицу предметов. Такое разложение позволяет найти скрытые факторы, описывающие как пользователей, так и предметы.

Часто используемые методы матричной факторизации в рекомендательных системах включают в себя:

1. **SVD (Singular Value Decomposition)**: Один из классических методов факторизации, который разбивает матрицу на три матрицы: матрицу пользователей, матрицу сингулярных значений и матрицу предметов.

2. **ALS (Alternating Least Squares)**: Это итеративный метод, который чередует оптимизацию матрицы пользователей и матрицы предметов.

Эти методы являются основой для многих алгоритмов рекомендательных систем, так как они позволяют сократить размерность данных и извлечь скрытые особенности, что помогает строить более эффективные модели рекомендаций.

In [4]:
def precision_at_k(result_recomendation, actual_recomendation, k=5):
    # Выбираем первые k рекомендаций из результата
    top_k = result_recomendation[:k]

    # Находим количество правильных рекомендаций среди первых k
    num_correct = len(set(top_k).intersection(actual_recomendation))

    # Вычисляем точность (precision) для первых k рекомендаций
    precision = num_correct / k

    # Возвращаем точность (precision) для заданного k
    return precision


In [5]:
# Чтение данных из CSV-файла
data = pd.read_csv('/content/drive/MyDrive/lesson_3/webinar/webianr_3/data/retail_train.csv')

# Приведение названий столбцов к нижнему регистру
data.columns = [col.lower() for col in data.columns]

# Переименование столбцов 'household_key' в 'user_id' и 'product_id' в 'item_id'
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'}, inplace=True)

# Определение размера тестовой выборки в неделях
test_size_weeks = 3

# Создание наборов данных для обучения и тестирования на основе временных интервалов
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Вывод первых 5 строк обучающего набора данных для проверки корректности
data_train.head(5)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [6]:
# Чтение данных из CSV-файла с характеристиками товаров
item_features = pd.read_csv('/content/drive/MyDrive/lesson_3/webinar/webianr_3/data/product.csv')

# Приведение названий столбцов к нижнему регистру
item_features.columns = [col.lower() for col in item_features.columns]

# Переименование столбца 'product_id' в 'item_id' для соответствия другим данным
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

# Вывод первых 3 строк таблицы с характеристиками товаров для проверки корректности
item_features.head(3)


,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,


In [7]:
# Получение уникальных значений из столбца 'department' в таблице характеристик товаров
unique_departments = item_features['department'].unique()

print(unique_departments)


['GROCERY' 'MISC. TRANS.' 'PASTRY' 'DRUG GM' 'MEAT-PCKGD' 'SEAFOOD-PCKGD'
 'PRODUCE' 'NUTRITION' 'DELI' 'COSMETICS' 'MEAT' 'FLORAL'
 'TRAVEL & LEISUR' 'SEAFOOD' 'MISC SALES TRAN' 'SALAD BAR' 'KIOSK-GAS'
 'ELECT &PLUMBING' 'GRO BAKERY' 'GM MERCH EXP' 'FROZEN GROCERY'
 'COUP/STR & MFG' 'SPIRITS' 'GARDEN CENTER' 'TOYS' 'CHARITABLE CONT'
 'RESTAURANT' 'RX' 'PROD-WHS SALES' 'MEAT-WHSE' 'DAIRY DELI' 'CHEF SHOPPE'
 'HBC' 'DELI/SNACK BAR' 'PORK' 'AUTOMOTIVE' 'VIDEO RENTAL' ' '
 'CNTRL/STORE SUP' 'HOUSEWARES' 'POSTAL CENTER' 'PHOTO' 'VIDEO'
 'PHARMACY SUPPLY']


In [8]:
# Группировка данных из data_test по столбцу 'user_id' и получение уникальных 'item_id' для каждого пользователя
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

# Переименование столбцов результирующего DataFrame в 'user_id' и 'actual'
result.columns = ['user_id', 'actual']

# Вывод первых двух строк для проверки результата
result.head(2)


,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [9]:
# Группировка данных из data_train по 'item_id' и вычисление общего количества проданных товаров
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()

# Переименование столбца 'quantity' в 'n_sold' для отражения количества проданных товаров
popularity.rename(columns={'quantity':'n_sold'}, inplace=True)

# Сортировка популярности товаров по количеству продаж и выбор топ-5000 самых популярных 'item_id'
top_5000 = popularity.sort_values(by='n_sold', ascending=False).iloc[:5000, :]['item_id'].tolist()

# Вывод первых 10 товаров из топ-5000 для проверки результата
top_5000[:10]


[6534178,
 6533889,
 6534166,
 6544236,
 1404121,
 397896,
 1426702,
 5703832,
 480014,
 5668996]

In [10]:
# Обновление значений 'item_id' в обучающем наборе данных data_train
# для товаров, которые не входят в топ-5000 популярных товаров
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999


In [11]:
# Вывод первых строк из data_train, где 'item_id' не входит в топ-5000 и были заменены на 999_999
data_train.loc[~data_train['item_id'].isin(top_5000)].head()


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
6,2375,26984851516,1,999999,1,1.57,364,-0.68,1642,1,0.0,0.0
8,2375,26984851516,1,999999,1,1.89,364,0.00,1642,1,0.0,0.0
9,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
10,2375,26984851516,1,999999,1,2.00,364,-0.79,1642,1,0.0,0.0
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0


In [12]:
# Создание матрицы пользователь-товар на основе обучающего набора данных data_train
# Используется функция pd.pivot_table(), которая группирует данные по 'user_id' и 'item_id',
# используя количество 'quantity' в качестве значения и подсчитывает количество с помощью агрегатной функции 'count'
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='quantity',
                                  aggfunc='count',
                                  fill_value=0  # Пропущенные значения заполняются нулями
                                  )

# Преобразование типа данных матрицы в тип float
user_item_matrix = user_item_matrix.astype(float)


In [13]:
user_item_matrix.head()

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Конвертация DataFrame в разреженную матрицу
sparse_user_item = csr_matrix(user_item_matrix)

# Преобразование разреженной матрицы обратно в плотную форму (numpy array) и вывод первых 9 строк
print(sparse_user_item.todense()[:9, :])


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# Получение массивов идентификаторов пользователей и товаров из индексов матрицы
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

# Создание массивов числовых идентификаторов для пользователей и товаров
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

# Создание словарей отображения исходных идентификаторов в числовые и обратно
id_to_itemid = dict(zip(matrix_itemids, itemids))  # Отображение числовых id -> item_id
id_to_userid = dict(zip(matrix_userids, userids))  # Отображение числовых id -> user_id

# Создание словарей отображения исходных item_id и user_id в числовые id
itemid_to_id = dict(zip(itemids, matrix_itemids))  # Отображение item_id -> числовый id
userid_to_id = dict(zip(userids, matrix_userids))  # Отображение user_id -> числовый id


In [16]:
%%time

# Создание модели ALS (Alternating Least Squares)
model = AlternatingLeastSquares(factors=64,  # Размерность векторов
                                regularization=0.05,  # Параметр регуляризации
                                iterations=15,  # Количество итераций
                                calculate_training_loss=True,  # Расчет функции потерь во время обучения
                                num_threads=1,  # Количество потоков
                                use_gpu=False  # Использование GPU (если доступно)
                                )

# Обучение модели ALS на транспонированной матрице item-user
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # Транспонированная item-user матрица
          show_progress=True  # Отображение прогресса обучения
          )

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 46.3 s, sys: 44 s, total: 1min 30s
Wall time: 1min 5s


In [17]:
# Генерация рекомендаций для конкретного пользователя
recs = model.recommend(userid=userid_to_id[2],  # Переводим исходный user_id в числовой id
                       user_items=csr_matrix(user_item_matrix).tocsr(),  # Матрица user-item на вход
                       N=5,  # Количество рекомендаций для генерации
                       filter_already_liked_items=False,  # Фильтрация рекомендаций, уже купленных пользователем
                       filter_items=None,  # Фильтр для товаров (по умолчанию, без фильтрации)
                       recalculate_user=True)  # Пересчитать представления пользователя


In [18]:
recs

[(4157, 1.0241892036084033),
 (2381, 0.9827925403872185),
 (3688, 0.9726765573792416),
 (4016, 0.9664080494155073),
 (3408, 0.9251200158969328)]

In [19]:
# Преобразование числовых идентификаторов товаров обратно в их исходные item_ids
[id_to_itemid[rec[0]] for rec in recs]


[5569230, 999999, 1106523, 1133018, 1082185]

In [20]:
def get_recommendations(user, model, N=5):
    # Получение рекомендаций для заданного пользователя с использованием модели
    # userid_to_id[user] переводит исходный user_id в числовой id
    res = [id_to_itemid[rec[0]] for rec in
                    model.recommend(userid=userid_to_id[user],
                                    user_items=sparse_user_item,   # Разреженная матрица user-item
                                    N=N,
                                    filter_already_liked_items=False,
                                    filter_items=None,
                                    recalculate_user=True)]
    return res


In [21]:
%%time

# Добавление столбца 'als_64', в котором хранятся рекомендации для каждого пользователя
result['als_64'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

# Применение функции precision_at_k к каждой строке DataFrame и расчет средней точности
precision_als_64 = result.apply(lambda row: precision_at_k(row['als_64'], row['actual']), axis=1).mean()

precision_als_64

CPU times: user 26.3 s, sys: 19.7 s, total: 45.9 s
Wall time: 30.8 s


0.16405484818805097

In [22]:
result.head(2)

,user_id,actual,als_64
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1100972, 1033142, 1024306, 832678]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 822407, 1092026, 951590, 1133018]"


###Embeddings

In [23]:
# Генерация матрицы предсказаний на основе факторов пользователей и товаров
# путем выполнения матричного умножения: векторы факторов пользователей (model.user_factors)
# на транспонированные векторы факторов товаров (model.item_factors.T)
fast_recs = model.user_factors @ model.item_factors.T

# Определение размерности полученной матрицы предсказаний
fast_recs.shape  # Возвращает количество строк (пользователей) и столбцов (товаров) в матрице


(2499, 5001)

In [24]:
fast_recs[:2]

array([[ 0.00584641,  0.10822037,  0.0060442 , ...,  0.0774478 ,
         0.06409188, -0.01733534],
       [ 0.00779703,  0.01702608, -0.01445082, ..., -0.05503125,
         0.05366453,  0.03315487]], dtype=float32)

In [25]:
%%time

# Генерация рекомендаций для всех пользователей с использованием обученной модели ALS
recommendations = model.recommend_all(N=5,  # Количество рекомендаций для каждого пользователя
                                      user_items=csr_matrix(user_item_matrix).tocsr(),  # Разреженная матрица user-item
                                      filter_already_liked_items=True,  # Фильтрация товаров, которые пользователь уже приобрел
                                      filter_items=None,  # Фильтр для товаров (по умолчанию, без фильтрации)
                                      recalculate_user=True,  # Пересчитать представления пользователя
                                      show_progress=True,  # Отображение прогресса генерации рекомендаций
                                      batch_size=500)  # Размер пакета для обработки данных (по умолчанию 1000)

recommendations


  0%|          | 0/2499 [00:00<?, ?it/s]

CPU times: user 27.7 s, sys: 20.5 s, total: 48.2 s
Wall time: 32.4 s


array([[3616, 2696,  192, 3031,  832],
       [2307, 2757, 4346, 2148, 2927],
       [2757,  558, 1923, 3557, 3388],
       ...,
       [4346, 2307, 3587,  558, 2148],
       [2847, 2757,  657,  300, 4157],
       [ 558, 3688, 4060, 2454, 1328]], dtype=int32)

###TF-IDF взвешивание

In [26]:
# Применение TF-IDF к транспонированной матрице user_item_matrix и транспонирование обратно
user_item_matrix = tfidf_weight(user_item_matrix.T).T

In [27]:
%%time

# Создание модели ALS с указанными параметрами
model = AlternatingLeastSquares(factors=64,  # Размерность векторов
                                regularization=0.05,  # Параметр регуляризации
                                iterations=15,  # Количество итераций
                                calculate_training_loss=True,  # Расчет функции потерь во время обучения
                                num_threads=1  # Количество потоков
                                )

# Обучение модели ALS на транспонированной матрице item-user
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # Транспонированная item-user матрица
          show_progress=True  # Отображение прогресса обучения
          )

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 921 ms, sys: 286 ms, total: 1.21 s
Wall time: 1.23 s


In [28]:
# Создание нового столбца 'als_tfidf' в DataFrame 'result', содержащего рекомендации для каждого пользователя
result['als_tfidf'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

# Применение функции precision_at_k к каждой строке DataFrame и расчет средней точности
precision_als_tfidf = result.apply(lambda row: precision_at_k(row['als_tfidf'], row['actual']), axis=1).mean()

precision_als_tfidf


0.16640548481880507

###BM25 взвешивание

In [29]:
# Замена значений в столбце 'item_id' в data_train на 999999 для товаров, не входящих в топ-5000
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

# Создание сводной таблицы user-item на основе обновленного data_train
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id',
                                  columns='item_id',
                                  values='quantity',  # Количество взаимодействий (в данном случае, количество приобретений)
                                  aggfunc='count',
                                  fill_value=0  # Заполнение пустых значений нулями
                                 )

# Приведение типа данных в матрице к типу float для использования в модели implicit
user_item_matrix = user_item_matrix.astype(float)

# Преобразование user_item_matrix в разреженную матрицу формата CSR для использования в модели implicit
sparse_user_item = csr_matrix(user_item_matrix).tocsr()


# Вывод первых трех строк матрицы user_item_matrix
user_item_matrix.head(3)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [31]:
%%time


# Создание модели ALS с указанными параметрами
model = AlternatingLeastSquares(
    factors=64,  # Количество факторов (размер скрытых векторов для пользователей и товаров)
    regularization=0.05,  # Параметр регуляризации для предотвращения переобучения
    iterations=15,  # Количество итераций обучения
    calculate_training_loss=True,  # Вычисление функции потерь во время обучения
    num_threads=10  # Количество потоков, используемых для обучения
)

# Обучение модели ALS на транспонированной матрице item-user
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # Транспонированная item-user матрица
          show_progress=True  # Отображение прогресса обучения
          )

# Создание нового столбца 'als_bm25' в DataFrame 'result', содержащего рекомендации для каждого пользователя
result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 26.4 s, sys: 19.2 s, total: 45.5 s
Wall time: 29.7 s


In [32]:
# Оценка средней точности рекомендаций, полученных из столбца 'als_bm25',
# по сравнению с фактическими покупками, содержащимися в столбце 'actual' для каждой строки в DataFrame 'result'
precision_als_bm25 = result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()

precision_als_bm25

0.1794319294809011

In [33]:
%%time

# Создание модели ALS с параметрами: factors=128, regularization=0.05, iterations=15 и num_threads=10
model = AlternatingLeastSquares(
    factors=128,  # Количество факторов (размер скрытых векторов для пользователей и товаров)
    regularization=0.05,  # Параметр регуляризации для предотвращения переобучения
    iterations=15,  # Количество итераций обучения
    calculate_training_loss=True,  # Вычисление функции потерь во время обучения
    num_threads=10  # Количество потоков, используемых для обучения
)

# Обучение модели ALS на транспонированной матрице item-user
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # Транспонированная item-user матрица
          show_progress=True  # Отображение прогресса обучения
)

# Создание нового столбца 'als_bm25_128' в DataFrame 'result', содержащего рекомендации для каждого пользователя
result['als_bm25_128'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 46.9 s, sys: 33.6 s, total: 1min 20s
Wall time: 52.8 s


In [34]:
# Оценка средней точности рекомендаций, предоставленных моделью ALS через столбец 'als_bm25_128',
# по сравнению с фактическими покупками, содержащимися в столбце 'actual' для каждой строки в DataFrame 'result'
precision_als_bm25_128 = result.apply(lambda row: precision_at_k(row['als_bm25_128'], row['actual']), axis=1).mean()

precision_als_bm25_128

0.19559255631733594

In [35]:
%%time

# Создание модели ALS с параметрами: factors=128, regularization=0.1, iterations=15 и num_threads=10
model = AlternatingLeastSquares(
    factors=128,  # Количество факторов (размер скрытых векторов для пользователей и товаров)
    regularization=0.1,  # Параметр регуляризации для предотвращения переобучения
    iterations=15,  # Количество итераций обучения
    calculate_training_loss=True,  # Вычисление функции потерь во время обучения
    num_threads=10  # Количество потоков, используемых для обучения
)

# Обучение модели ALS на транспонированной матрице item-user
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # Транспонированная item-user матрица
          show_progress=True  # Отображение прогресса обучения
)

# Создание нового столбца 'als_bm25_256' в DataFrame 'result', содержащего рекомендации для каждого пользователя
result['als_bm25_128_reg'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))


  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 47.3 s, sys: 34.3 s, total: 1min 21s
Wall time: 53 s


In [36]:
# Оценка средней точности рекомендаций, предоставленных моделью ALS через столбец 'als_bm25_128_reg',
# по сравнению с фактическими покупками, содержащимися в столбце 'actual' для каждой строки в DataFrame 'result'
precision_als_bm25_128_reg = result.apply(lambda row: precision_at_k(row['als_bm25_128_reg'], row['actual']), axis=1).mean()

precision_als_bm25_128_reg

0.19696376101860924

In [37]:
# Вычисление процентного изменения между precision_als_64 и precision_als_bm25_128
percent_change = (1 - precision_als_64 / precision_als_bm25_128_reg) * 100

# Вывод результата
print(f"Процентное улучшение точности модели ALS с методом BM25 и factors=128 относительно ALS (factors=64): {abs(percent_change):.2f}%")

Процентное улучшение точности модели ALS с методом BM25 и factors=128 относительно ALS (factors=64): 16.71%


###Вывод:

___

На основании предоставленных данных можно сделать вывод о значительном улучшении точности модели ALS с методом BM25 и параметром factors=128 по сравнению с ALS, где использовались параметры factors=64. Процентное улучшение точности составляет **16.71%.**

Это показывает, что модель ALS, использующая метод BM25 и большее количество факторов (128), предоставляет более точные рекомендации или предсказания по сравнению с моделью ALS, где использовались меньшее количество факторов (64). Такое значительное улучшение может быть важным при решении задачи рекомендательной системы или анализе данных.

Кроме того, изменение параметра regularization с 0.05 на 0.1 в модели ALS с методом BM25 и factors=128 также привело к улучшению уровня точности рекомендаций. Увеличение значения параметра regularization внесло свой вклад в повышение точности модели.

Таким образом, увеличение значения regularization в данном случае способствует улучшению точности рекомендательной модели ALS с использованием метода BM25 и большего количества факторов (128) по сравнению с значениями regularization=0.05. Это указывает на то, что более сильная регуляризация (в данном случае, regularization=0.1) в сочетании с увеличением количества факторов способствует улучшению качества рекомендаций модели.

___